In [1]:
from urllib.request import urlopen
import requests
import re
from bs4 import BeautifulSoup, Doctype
from bs4 import Comment
from selenium import webdriver

## Descargar nuestros datos

In [2]:
def all_pages(num):
    url=f'https://jobcube.com/Data-Science-jobs?pg={num}'
    response=requests.get(url)
    page = BeautifulSoup(response.text, 'lxml')
    return page

In [3]:
id_page = [i for i in range(1,61)]
link_page=[all_pages(i) for i in id_page]

In [4]:
urls=[]
for i in range(len(link_page)):
    links = link_page[i].find_all('h2','p-job_title')
    for i in links:
        children = i.find("a",'s-biggerLink')
        urls.append(children.get('href'))

In [5]:
len(urls)

1200

In [13]:
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

def links_htmls(k):
    r = requests.get(urls[k], headers=HEADERS)
    soup=BeautifulSoup(r.text,'lxml')
    
    [x.extract() for x in soup.find_all('svg')]
    [x.extract() for x in soup.find_all('img')]

    title = soup.find("title")
    p = soup.find("p",class_='p-detail_head_title')
    description = soup.find("div",class_='p-detail_content')

    head = f"<head>{title}</head>"
    h1=f'<h1>{p.get_text()}</h1>'
    body = f"<body>{h1}{description}</body>"
    html_output = f"<html>{head}{body}</html>"

    html_output= re.sub(r'[\t\r\n]', '', html_output)

    with open(f'/media/richi/1C0C-8070/ItAcademyDS/sprint12/jobs_usa/jobcube{k}.html', 'w',encoding='utf-8') as f:
        f.write(str(html_output))

In [19]:
for i in range(0,len(urls)):
    links_htmls(i)  

```python
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin

##### Web scrapper for infinite scrolling page #####
driver = webdriver.Chrome()
driver.get("https://jooble.org/SearchResult?rgns=United%20States&ukw=data%20science")
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break 

##### Extract Reddit URLs #####
link = driver.find_elements_by_xpath('//a[@data-testid="jobTitle"]')
links = [elem.get_attribute('href') for elem in link]
```

```python
r = requests.get(urls[0], headers=HEADERS)
soup=BeautifulSoup(r.text,'lxml')
    
[x.extract() for x in soup.find_all('svg')]
[x.extract() for x in soup.find_all('img')]


title = soup.find("title")
    
p = soup.find("p",class_='p-detail_head_title')
    
description = soup.find("div",class_='p-detail_content')
    
head = f"<head>{title}</head>"
h1=f'<h1>{p.get_text()}</h1>'
body = f"<body>{h1}{description}</body>"
html_output = f"<html>{head}{body}</html>"

html_output= re.sub(r'[\t\r\n]', '', html_output)

with open('jobcube.html', 'w',encoding='utf-8') as f:
    f.write(str(html_output))
```